In [3]:
import os
import json
import shutil
from pathlib import Path

import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm

SUPPORTED_EXTS = [".jpg", ".jpeg", ".png", ".bmp", ".webp"]


In [5]:
def find_image_for_id(images_dir: Path, img_id: str):
    """Try exact filename with common extensions, else wildcard id.*"""
    for ext in SUPPORTED_EXTS:
        p = images_dir / f"{img_id}{ext}"
        if p.exists():
            return p
    # also try case-insensitive / any file starting with id + dot
    candidates = list(images_dir.glob(f"{img_id}.*"))
    return candidates[0] if candidates else None
